In [49]:
import requests
from bs4 import BeautifulSoup
import tqdm
import time
import numpy as np
import pandas as pd
import nltk   
import unicodedata
from html.parser import HTMLParser
import re
from tqdm import tqdm
import glob
from utils import *
import pathlib

dataset_name = "dataset_ricardoeletro.csv"
dataset_path = "data/ricardoeletro/*"

In [158]:
pages_url = glob.glob(dataset_path)
X_data, y_data = [], []
for page in tqdm(pages_url):

    stem  = pathlib.Path(page).stem
    y     = int(stem.split('_')[1])
    with open(page,"r") as f:
        lines = f.readlines()
        doc   = '\n'.join(lines)

    s = BeautifulSoup(doc, "html.parser")
    
    head_text, att = '', ''
    if s.head is not None:
        head_text = s.head.title.find_all(text=True)
    
    if len(s.find_all("table")) > 0:              
        table       = max(s.find_all("table"),key=len)
        np_table    = (html_to_np(table))

        if len(np_table.shape)>1 and  np_table.shape[1] > 1:
            att = np_table[:,0]#, np_table[:,1]
        
    X = ' '.join(head_text) + ' '+ ' '.join(att)

    regex = re.compile('[^a-zA-Z ]')
    X     = regex.sub('', X)
    X     = [ word.lower() for word in  X.split(' ') if len(word) > 2]
    X     = sorted(list(set(X)))
    X_data.append(X)
    y_data.append(y)
header = sorted(list(set(sum(X_data,[]))))

100%|██████████| 203/203 [00:06<00:00, 30.95it/s]


In [159]:
X_pd = [[header[i] in X_data[j] for i in range(len(header))] for j in range(len(X_data))]

In [162]:
dataset = pd.DataFrame(X_pd, columns=header)
dataset['y'] = y_data
dataset.head()


,abreviada,acelerometro,acesso,acessorios,acessos,action,air,alarme,alcatel,alerta,...,wificonnection,xbox,xiaomi,youtube,zakl,zbkl,zckl,zenfone,zoom,y
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,1
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,0
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,0
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,0
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,1


In [164]:
dataset.to_csv("clf_ricardoeletro.csv",index=False)